In [1]:
###CONFIGURAÇÕES INICIAIS###

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, date_format, dayofweek, month, year, quarter, concat, lit, to_date, datediff
from datetime import datetime, timedelta
import holidays


# Inicializando a sessão do Spark
spark = SparkSession.builder \
    .appName("Silver") \
    .getOrCreate()

# Caminho para salvar o arquivo
file_path = 'C:/Users/yurib/pjf-engenharia-de-dados/algo/'

In [ ]:
### CARREGANDO ARQUIVOS DA CAMADA BRONZE ###

# Carregando dimensões e fato da camada Bronze diretamente do Spark
datas = spark.table('dim_data')
pedidos = spark.table('pedidos')
localizacoes = spark.table('localizacoes')

In [3]:
### CRIANDO TRANSACAO ###

dim_transacao = pedidos.select('idcliente', 'SK_Pedidos', 'idvendedor', 'pagamento')

In [4]:
### CRIANDO ENTREGAS ###

entregas = pedidos.select('SK_Pedidos', 'cepcliente', 'cepvendedor')

# Join para destino
temp_destino = entregas.join(localizacoes.withColumnRenamed('cep', 'cepcliente'), on='cepcliente', how='left')
entregas = temp_destino.withColumnRenamed('SK_Localizacoes', 'en_destino').drop('cepcliente', 'latitude', 'longitude')

# Join para origem
temp_origem = entregas.join(localizacoes.withColumnRenamed('cep', 'cepvendedor'), on='cepvendedor', how='left')
entregas = temp_origem.withColumnRenamed('SK_Localizacoes', 'en_origem').drop('cepvendedor', 'latitude', 'longitude')

In [5]:
### TRATANDO DADOS DE PEDIDOS ###

# Calculando volume e massa
pedidos = pedidos.withColumn('volume', (col('comprimento_cm') * col('altura_cm') * col('largura_cm')) / (100 * 100 * 100))
pedidos = pedidos.withColumn('massa', col('peso_g') / 1000)

# Renomeando coluna
pedidos = pedidos.withColumnRenamed('id_produto', 'idproduto')

In [6]:
### CRIANDO FATO DE PEDIDOS ###

# Selecionando e renomeando colunas
fato = pedidos.select('SK_Pedidos', 'iditem', 'preco', 'frete', 'datacompra', 'datacliente', 'prazo', 'volume', 'massa')

# Calculando dias de entrega e dias de prazo
fato = fato.withColumn('ped_diasentrega', datediff(col('datacliente'), col('datacompra')))
fato = fato.withColumn('ped_diasprazo', datediff(col('prazo'), col('datacompra')))

# Selecionando colunas finais
final = fato.select('SK_Pedidos', 'preco', 'frete', 'volume', 'massa', 'ped_diasentrega', 'ped_diasprazo', 'datacompra')

# Tratando data
final = final.withColumn('data_ref', to_date(col('datacompra')))
final = final.join(datas, final['data_ref'] == datas['dt_data'], 'left')
final = final.withColumn('id_data', col('sk_data')).drop('sk_data')

In [8]:
### SALVANDO EM SPARK NA CAMADA SILVER ###

entregas.write.mode('overwrite').saveAsTable('dim_entregas')
dim_transacao.write.mode('overwrite').saveAsTable('dim_transacao')
localizacoes.write.mode('overwrite').saveAsTable('dim_localizacoes')
final.write.mode('overwrite').saveAsTable('fato_pedidos')
datas.write.mode('overwrite').saveAsTable('dim_data')

In [ ]:
# Finalizando a sessão Spark
spark.stop()